## GRCh38

GRCh38 is a human reference genome, meaning although it is be assembled by sequencing the DNA of various people it is thought to be representative of human DNA. The data was downloaded from [Human Genome Resources at NCBI](https://www.ncbi.nlm.nih.gov/projects/genome/guide/human/)

In [1]:
# pip3 install biopython
from Bio import SeqIO

In [2]:
gen = SeqIO.parse('GRCh38_latest_genomic.fna', 'fasta')
# The file is 4 GB
records = list(gen)

In [3]:
print(f'There are {len(records)} sequences')

There are 594 sequences


In [4]:
for seq_record in records:
    print(f'DESCRIPTION: {seq_record.description}')
    print(f'LENGTH: {len(seq_record)}')
    print()

DESCRIPTION: NC_000001.11 Homo sapiens chromosome 1, GRCh38.p12 Primary Assembly
LENGTH: 248956422

DESCRIPTION: NT_187361.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p12 Primary Assembly HSCHR1_CTG1_UNLOCALIZED
LENGTH: 175055

DESCRIPTION: NT_187362.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p12 Primary Assembly HSCHR1_CTG2_UNLOCALIZED
LENGTH: 32032

DESCRIPTION: NT_187363.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p12 Primary Assembly HSCHR1_CTG3_UNLOCALIZED
LENGTH: 127682

DESCRIPTION: NT_187364.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p12 Primary Assembly HSCHR1_CTG4_UNLOCALIZED
LENGTH: 66860

DESCRIPTION: NT_187365.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p12 Primary Assembly HSCHR1_CTG5_UNLOCALIZED
LENGTH: 40176

DESCRIPTION: NT_187366.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p12 Primary Assembly HSCHR1_CTG6_UNLOCALIZED
LENGTH: 42210

DESCRIPTIO

There are lot of sequences in the file because most are additional data. The defintions are stated [here on the NCBI Assembly Terminology website](https://www.ncbi.nlm.nih.gov/grc/help/definitions/). I believe the gist is that geneticists are unsure of which chromosomes the extra sequences belong to, unsure where they fit it on the chromosome, etc. Or they are alternatives for specific segments of a chromosome. For our purposes though we can just look at the main chromosomes.

In [5]:
chromosomes = []
for seq_record in records:
    if "scaffold" not in seq_record.description and "patch" not in seq_record.description:
        chromosomes.append(seq_record)

In [6]:
print(len(chromosomes))

25


In [7]:
for seq_record in chromosomes:
    print(f'DESCRIPTION: {seq_record.description}')
    print(f'LENGTH: {len(seq_record)}')
    print()

DESCRIPTION: NC_000001.11 Homo sapiens chromosome 1, GRCh38.p12 Primary Assembly
LENGTH: 248956422

DESCRIPTION: NC_000002.12 Homo sapiens chromosome 2, GRCh38.p12 Primary Assembly
LENGTH: 242193529

DESCRIPTION: NC_000003.12 Homo sapiens chromosome 3, GRCh38.p12 Primary Assembly
LENGTH: 198295559

DESCRIPTION: NC_000004.12 Homo sapiens chromosome 4, GRCh38.p12 Primary Assembly
LENGTH: 190214555

DESCRIPTION: NC_000005.10 Homo sapiens chromosome 5, GRCh38.p12 Primary Assembly
LENGTH: 181538259

DESCRIPTION: NC_000006.12 Homo sapiens chromosome 6, GRCh38.p12 Primary Assembly
LENGTH: 170805979

DESCRIPTION: NC_000007.14 Homo sapiens chromosome 7, GRCh38.p12 Primary Assembly
LENGTH: 159345973

DESCRIPTION: NC_000008.11 Homo sapiens chromosome 8, GRCh38.p12 Primary Assembly
LENGTH: 145138636

DESCRIPTION: NC_000009.12 Homo sapiens chromosome 9, GRCh38.p12 Primary Assembly
LENGTH: 138394717

DESCRIPTION: NC_000010.11 Homo sapiens chromosome 10, GRCh38.p12 Primary Assembly
LENGTH: 133797422


I think the structure of GRCh38 is pretty light so having everything as BioPython classes doesn't really help. Might as well convert everything to a string.

In [15]:
last_names = ['chrX', 'chrY', 'mito']
def seq_name(i):
    assert i >= 0 and i < 25
    if i < 22:
        return f'chr{i+1}'
    else:
        return last_names[i-22]

In [16]:
for (i, seq_record) in enumerate(chromosomes):
    sequence = str(seq_record.seq)
    text_file = open(f'grch38_dna/{seq_name(i)}.txt', 'w')
    text_file.write(sequence)
    text_file.close()

In [4]:
print(seq_record)

ID: NC_000001.11
Name: NC_000001.11
Description: NC_000001.11 Homo sapiens chromosome 1, GRCh38.p12 Primary Assembly
Number of features: 0
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN', SingleLetterAlphabet())


In [5]:
seq = seq_record.seq

In [6]:
len(seq)

248956422

In [7]:
seq[100000:1000500]

Seq('ACTAAGCACACAGAGAATAATGTCTAGAATCTGAGTGCCATGTTATCAAATTGT...AGG', SingleLetterAlphabet())

In [8]:
seq.count('N')

18475408

In [9]:
import numpy as np

In [10]:
PATTERN_LENGTH = 1000
TRIALS = 50
rand_indexes = np.random.randint(len(seq) - PATTERN_LENGTH + 1, size = TRIALS)
rand_seqs = [seq[index : index + PATTERN_LENGTH] for index in rand_indexes]

In [11]:
import time

In [12]:
# https://github.com/python/cpython/blob/master/Objects/stringlib/fastsearch.h#L5

average_time = 0
for (pattern, index) in zip(rand_seqs, rand_indexes):
    start = time.clock()
    found_index = seq.find(pattern)
    end = time.clock()
    time_elapsed = end - start
    average_time += time_elapsed
    print("Pattern starting at index {}. Found match at index {}. Took {}".format(index, found_index, time_elapsed))
print("Average time: {}".format(average_time / TRIALS))

Pattern starting at index 99702246. Found match at index 99702246. Took 0.3440430000000001
Pattern starting at index 248257787. Found match at index 248257787. Took 0.7676189999999998
Pattern starting at index 106981401. Found match at index 106981401. Took 0.22026699999999977
Pattern starting at index 83230124. Found match at index 83230124. Took 0.29790700000000037
Pattern starting at index 207616487. Found match at index 207616487. Took 0.7086350000000001
Pattern starting at index 143083843. Found match at index 0. Took 1.5999999999571912e-05
Pattern starting at index 27889613. Found match at index 27889613. Took 0.06140900000000027
Pattern starting at index 316556. Found match at index 0. Took 1.4999999999432134e-05
Pattern starting at index 215194961. Found match at index 215194961. Took 0.5609480000000007
Pattern starting at index 69591945. Found match at index 69591945. Took 0.20468200000000003
Pattern starting at index 74235958. Found match at index 74235958. Took 0.29985000000

In [18]:
dna_string = str(seq)
len(dna_string)

248956422

In [17]:
counts = {}
for base in dna_string:
    if base not in counts:
        counts[base] = 0
    counts[base] += 1
print(counts)

{'N': 18475408, 't': 25922550, 'a': 25810086, 'c': 16788841, 'g': 16855483, 'C': 31266202, 'G': 31256045, 'T': 41321614, 'A': 41260191, 'M': 1, 'R': 1}
